In [8]:
import os, gc, torch

import numpy as np
import pandas as pd
# from models.load import TrainedModels

from utils.eval import save_iou_results
from utils.engine import xami_evaluate, get_iou_types
from models.load import get_trained_model
from utils.coco_eval import get_eval_params_dict
from data.datasets import collate_fn
from our_radiologist.load import get_anns
from utils.coco_utils import get_cocos, get_coco_api_from_dataset
from utils.eval import get_ap_ar
from utils.print import print_title
from utils.init import reproducibility, clean_memory_get_device
from data.load import get_datasets, get_dataloaders
from data.constants import XAMI_MIMIC_PATH, DEFAULT_REFLACX_LABEL_COLS
from utils.constants import full_iou_thrs, iou_thrs_5to95
from data.load  import seed_worker, get_dataloader_g
from tqdm import tqdm

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore")


%matplotlib inline

In [9]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CPU]


In [10]:
from enum import Enum

class TrainedModels(Enum):
    # CXR_images_pupil="val_ar_0_3786_ap_0_1456_test_ar_0_4015_ap_0_1718_epoch26_09-19-2022 02-32-59_CXR_images_pupil"
    # CXR_images_resnet= "val_ar_0_6295_ap_0_2233_test_ar_0_6295_ap_0_2365_epoch13_10-05-2022 05-28-54_CXR_images_resnet"
    # CXR_images_fixations_multiply= "val_ar_0_2275_ap_0_1134_test_ar_0_2464_ap_0_1187_epoch16_10-07-2022 09-07-36_CXR_images__fixations_multiply"
    # CXR_images_1third= "val_ar_0_1143_ap_0_0292_test_ar_0_1206_ap_0_0349_epoch14_09-22-2022 16-58-38_CXR_images_fixations_1third"
    # CXR_images_fixations_mobilenet = "val_ar_0_0552_ap_0_0182_test_ar_0_0622_ap_0_0138_epoch11_10-09-2022 03-42-33_CXR_images_fixations_mobilenet"
    # CXR_images_fixations_resnet = "val_ar_0_2876_ap_0_1181_test_ar_0_3026_ap_0_1391_epoch23_10-11-2022 06-58-51_CXR_images_fixations_resnet"
    # CXR_images_pupil_multiply = "val_ar_0_3076_ap_0_1231_test_ar_0_3568_ap_0_1647_epoch9_10-16-2022 01-28-41_CXR_images_pupil_multiply"
    # CXR_images_fixations_1third_multiply = "val_ar_0_1789_ap_0_0894_test_ar_0_1432_ap_0_0704_epoch26_10-18-2022 21-24-52_CXR_images_fixations_1third_multiply"
    CXR_images_fixations_2third_multiply = "val_ar_0_1052_ap_0_0364_test_ar_0_0927_ap_0_0367_epoch4_10-19-2022 16-24-41_CXR_images_fixations_2third_multiply"


    



In [11]:
normal_iou_thrs = iou_thrs_5to95
all_range_iou_thrs = full_iou_thrs
score_thresholds = [0.05]

In [12]:
for select_model in tqdm([
    TrainedModels.CXR_images_fixations_2third_multiply,
]):

    model, train_info, _, _ = get_trained_model(
            select_model,
            DEFAULT_REFLACX_LABEL_COLS,
            device,
            rpn_nms_thresh=0.3,
            box_detections_per_img=10,
            box_nms_thresh=0.2,
            rpn_score_thresh=0.0,
            box_score_thresh=0.5,
        )

    print(train_info)

  0%|          | 0/1 [00:00<?, ?it/s]

Using pretrained backbone. resnet18
Using pretrained backbone. resnet18
CXR_images_fixations_2third_multiply will use mask, [64] layers.
Using SGD as optimizer with lr=0.001


100%|██████████| 1/1 [00:03<00:00,  3.28s/it]

========================================For Training [CXR_images_fixations_2third_multiply]========================================
ModelSetup(name='CXR_images_fixations_2third_multiply', use_heatmaps=True, with_fixations=False, with_pupil=False, with_1st_third_fixations=False, with_2nd_third_fixations=True, with_rad_silence=False, with_rad_speaking=False, save_early_stop_model=True, record_training_performance=True, backbone='resnet18', optimiser='sgd', lr=0.001, weight_decay=1e-05, image_backbone_pretrained=True, heatmap_backbone_pretrained=True, image_size=512, backbone_out_channels=64, batch_size=4, warmup_epochs=0, lr_scheduler='ReduceLROnPlateau', reduceLROnPlateau_factor=0.1, reduceLROnPlateau_patience=999, reduceLROnPlateau_full_stop=True, multiStepLR_milestones=100, multiStepLR_gamma=0.1, representation_size=64, mask_hidden_layers=64, using_fpn=False, use_mask=True, fuse_conv_channels=64, box_head_dropout_rate=0, fuse_depth=0, fusion_strategy='multiply', fusion_residule=False,

# Run evaluation.

In [13]:
for select_model in tqdm([
    TrainedModels.CXR_images_fixations_2third_multiply,
    # TrainedModels.CXR_images_fixations_resnet,
]):

    for score_thrs in score_thresholds:

        model, train_info, _, _ = get_trained_model(
            select_model,
            DEFAULT_REFLACX_LABEL_COLS,
            device,
            rpn_nms_thresh=0.3,
            box_detections_per_img=10,
            box_nms_thresh=0.2,
            rpn_score_thresh=0.0,
            box_score_thresh=score_thrs,
        )

        model.eval()

        iou_types = get_iou_types(model, train_info.model_setup)

        model_setup = train_info.model_setup
        dataset_params_dict = {
        "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
        "bbox_to_mask": model_setup.use_mask,
        "labels_cols": DEFAULT_REFLACX_LABEL_COLS,
        "with_fixations": True
        }


        detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
            dataset_params_dict=dataset_params_dict,
        )

        train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
            train_dataset, val_dataset, test_dataset, batch_size=4,
        )

        train_coco, val_coco, test_coco = get_cocos(
            train_dataloader, val_dataloader, test_dataloader
        )

        normal_eval_params_dict = get_eval_params_dict(
            detect_eval_dataset, iou_thrs=normal_iou_thrs,
        )

        all_range_eval_params_dict = get_eval_params_dict(
            detect_eval_dataset, iou_thrs=all_range_iou_thrs,
        )

        all_cat_ids = [None] + [
            detect_eval_dataset.disease_to_idx(d)
            for d in detect_eval_dataset.labels_cols
        ]

        for cat_id in all_cat_ids:
            cat_ids = (
                [
                    detect_eval_dataset.disease_to_idx(d)
                    for d in detect_eval_dataset.labels_cols
                ]
                if cat_id is None
                else [cat_id]
            )

            if not (cat_ids is None):
                all_range_eval_params_dict["bbox"].catIds = cat_ids
                all_range_eval_params_dict["segm"].catIds = cat_ids

            train_evaluator, _ = xami_evaluate(
                model,
                train_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=train_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            test_evaluator, _ = xami_evaluate(
                model,
                test_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=test_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            val_evaluator, _ = xami_evaluate(
                model,
                val_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=val_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            if cat_id is None:
                disease_str = "all"
            else:
                disease_str = detect_eval_dataset.label_idx_to_disease(cat_id)

            save_iou_results(
                train_evaluator,
                f"train_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )
            save_iou_results(
                test_evaluator,
                f"test_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )
            save_iou_results(
                val_evaluator,
                f"val_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )

            train_ap_ar = get_ap_ar(
                train_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )

            test_ap_ar = get_ap_ar(
                test_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )
            val_ap_ar = get_ap_ar(
                val_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )

            df = pd.DataFrame(
                [
                    {
                        "dataset": "train",
                        f"AP@[IoBB = 0.50]": train_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": train_ap_ar["ar"],
                    },
                    {
                        "dataset": "test",
                        f"AP@[IoBB = 0.50]": test_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": test_ap_ar["ar"],
                    },
                    {
                        "dataset": "val",
                        f"AP@[IoBB = 0.50]": val_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": val_ap_ar["ar"],
                    },
                ]
            )

            df.to_csv(
                os.path.join(
                    "eval_results",
                    f"{select_model.value}_{disease_str}_score_thrs{score_thrs}.csv",
                )
            )
            print_title(disease_str)
            print(df)


  0%|          | 0/1 [00:00<?, ?it/s]

Using pretrained backbone. resnet18
Using pretrained backbone. resnet18
CXR_images_fixations_2third_multiply will use mask, [64] layers.
Using SGD as optimizer with lr=0.001
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
Evaluation:  [  0/531]  eta: 0:58:24  loss: 0.4867 (0.4867)  loss_classifier: 0.0084 (0.0084)  loss_box_reg: 0.0072 (0.0072)  loss_mask: 0.4417 (0.4417)  loss_objectness: 0.0274 (0.0274)  loss_rpn_box_reg: 0.0020 (0.0020)  model_time: 4.5448 (4.5448)  evaluator_time: 0.0030 (0.0030)  time: 6.6003  data: 2.0525
Evaluation:  [100/531]  eta: 0:48:54  loss: 0.5216 (0.4824)  loss_classifier: 0.0167 (0.0216)  loss_box_reg: 0.0090 (0.0122)  loss_mask: 0.4333 (0.4016)  loss_objectness: 0.0316 (0.0426)  loss_rpn_box_reg: 0.0030 (0.0044)  model_time: 4.5388 (4.5927)  evaluator_time: 0.2673 (0.2221)  time: 6.8752  data: 1.9018
Evaluation:  [200/531]  eta: 0:36:15 

100%|██████████| 1/1 [8:25:55<00:00, 30355.65s/it]

  dataset  AP@[IoBB = 0.50]  AR@[IoBB = 0.50]
0   train          0.009169          0.046229
1    test          0.001320          0.013699
2     val          0.010891          0.039604


In [1]:
from data.datasets import ReflacxDataset, collate_fn
import pandas as pd
from data.constants import DEFAULT_REFLACX_LABEL_COLS, XAMI_MIMIC_PATH

box_df = ReflacxDataset(XAMI_MIMIC_PATH)
fix = 'E:\XAMI-MIMIC\patient_15752803\REFLACX\P300R227438\\anomaly_location_ellipses.csv'
box_df.generate_bboxes_df(pd.read_csv(fix))

c:\Users\VIMMI\Desktop\multimodal-abnormalities-detection\data\datasets.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxes_df[k] = ellipse_df[
c:\Users\VIMMI\Desktop\multimodal-abnormalities-detection\data\datasets.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boxes_df[k] = ellipse_df[
c:\Users\VIMMI\Desktop\multimodal-abnormalities-detection\data\datasets.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

,xmin,ymin,xmax,ymax,certainty,label
0,140,1957,1251,2272,4.0,Atelectasis
1,1641,1995,2533,2640,4.0,Atelectasis


1. One ellipse can have more than one label.
2. Each label can have different certainties.
3. One label can be asigned to more than one ellipse, for the same patient.